In [20]:
import geopandas as gpd
import numpy as np

In [21]:
gdf = gpd.read_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_final.shp")
print(gdf)

      fid              fname  year    id type                sdate  \
0      29  Abrantes_09082017  2017   331    z  2017-08-09 18:00:00   
1      29  Abrantes_09082017  2017   334    p  2017-08-09 18:00:00   
2      29  Abrantes_09082017  2017   335    p  2017-08-09 23:00:00   
3      29  Abrantes_09082017  2017   336    p  2017-08-09 23:00:00   
4      29  Abrantes_09082017  2017   333    z  2017-08-10 12:00:00   
...   ...                ...   ...   ...  ...                  ...   
3350  135   Vinhais_01092024  2024  2795    p                  NaT   
3351  135   Vinhais_01092024  2024  2796    p                  NaT   
3352  135   Vinhais_01092024  2024  2797    p                  NaT   
3353  135   Vinhais_01092024  2024  2798    p                  NaT   
3354  135   Vinhais_01092024  2024  2799    p                  NaT   

                    edate      inidoy      enddoy    source  ...  LFC_hPa_av  \
0                     NaT   -1.000000   -1.000000      fops  ...         NaN   

In [22]:
UTILISAR_SO_BPERIOD1 = False

if UTILISAR_SO_BPERIOD1:
    gdf = gdf[gdf['burn_perio'] == 1]
    print(gdf)

In [23]:
# Listas de variáveis por valor nulo especial
vars_com_no_value_1 = ['Cin_av']

vars_com_no_value_minus1 = [
    'CBH_m_av', 'LFC_hPa_av', 'CCL_hPa_av', 'EL_m_av',
    'Recirc', 'CircVar'
    ]

vars_com_no_value_minus999 = [
    'BLH_m_rt', 'gT_s_9_av', 'gT_9_8_av', 'gT_8_7_av', 'gT_7_5_av', 'gT_5_3_av',
    'wSv_9_av', 'wSdir_9_av', 'wSv_7_av', 'wSdir_7_av',
    'wSv_5_av', 'wSdir_5_av', 'wSv_1_av', 'wSdir_1_av'
]

# Substituir valores especiais por NaN conforme o tipo
gdf[vars_com_no_value_1] = gdf[vars_com_no_value_1].replace(1, float("nan"))

gdf[vars_com_no_value_minus1] = gdf[vars_com_no_value_minus1].replace(-1, float("nan"))

gdf[vars_com_no_value_minus999] = gdf[vars_com_no_value_minus999].replace(-999, float("nan"))


In [24]:
gdf = gdf[~gdf['fire_rank'].isin([-4, -3, -2, -1, 3])]
gdf = gdf.drop(columns=['fire_rank'])
print(gdf)

      fid              fname  year    id type                sdate  \
1      29  Abrantes_09082017  2017   334    p  2017-08-09 18:00:00   
2      29  Abrantes_09082017  2017   335    p  2017-08-09 23:00:00   
5      29  Abrantes_09082017  2017   338    p  2017-08-10 12:00:00   
8      29  Abrantes_09082017  2017   340    p  2017-08-10 14:30:00   
23    113  Abrantes_29082023  2023  2669    p  2023-08-29 17:20:00   
...   ...                ...   ...   ...  ...                  ...   
3342  135   Vinhais_01092024  2024  2788    p  2024-09-01 18:00:00   
3345  135   Vinhais_01092024  2024  2791    p  2024-09-01 18:50:00   
3346  135   Vinhais_01092024  2024  2792    p  2024-09-02 01:00:00   
3348  135   Vinhais_01092024  2024  2793    p  2024-09-02 15:00:00   
3349  135   Vinhais_01092024  2024  2794    p  2024-09-02 16:50:00   

                    edate      inidoy      enddoy    source  ...  LCL_m_av  \
1     2017-08-09 23:00:00  221.750000  221.958333      fops  ...  2.187129   
2  

In [25]:
gdf.to_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_short.shp")

In [26]:
import pandas as pd

# garantir datetime
gdf['sdate'] = pd.to_datetime(gdf['sdate'])

# intervalos a remover
mask1 = (gdf['sdate'] >= '2017-06-17') & (gdf['sdate'] <= '2017-06-21')
mask2 = (gdf['sdate'] >= '2017-10-15') & (gdf['sdate'] <= '2017-10-16')

mask_remove =  mask2

# contar quantas serão retiradas
num_removidas = gdf[mask_remove].shape[0]

# fazer a remoção
gdf_filtrado = gdf[~mask_remove]

# contar quantas sobram
num_sobra = gdf_filtrado.shape[0]

num_removidas, num_sobra

(119, 1058)

In [27]:
# 2. Variáveis a excluir
vars_excluir = [
    'fid', 'fname', 'year', 'id', 'type', 'sdate', 'edate',
    'source', 'zp_link', 'burn_perio', 'area', 'growth_rat', 'ros_i',
    'spdir_i', 'spdir_p', 'int_i', 'int_p', 'duration_i', 'qc', 'land_use_d'
]

# 3. Remover colunas indesejadas (se existirem)
gdf = gdf.drop(columns=[c for c in vars_excluir if c in gdf.columns])
gdf_filtrado = gdf_filtrado.drop(columns=[c for c in vars_excluir if c in gdf_filtrado.columns])

for i, coluna in enumerate(gdf.columns, 1):
    print(f"{coluna}")

# 4. remover -1s
gdf.replace(-1, np.nan, inplace=True)
gdf_filtrado.replace(-1, np.nan, inplace=True)

inidoy
enddoy
ros_p
duration_p
elev_av
aspect_av
landform
land_use
1_3y_fir_p
3_8y_fir_p
8_ny_fir_p
fuel_model
f_load_av
sW_1m_av
sW_3m_av
sW_7_av
sW_28_av
sW_100_av
sW_289_av
t_2m_C_av
d_2m_C_av
rh_2m_av
VPD_Pa_av
sP_hPa_av
gp_m2s2_av
dfmc_av
HDW_av
Haines_av
FWI_12h_av
DC_12h_av
FFMC_12h_a
wv10_kh_av
wdir10_av
wv100_k_av
wdir100_av
Recirc
CircVar
t_950_av
t_850_av
t_700_av
t_500_av
t_300_av
rh_950_av
rh_850_av
rh_700_av
rh_500_av
rh_300_av
wv_950_av
wv_850_av
wv_700_av
wv_500_av
wv_300_av
wdi_950_av
wdi_850_av
wdi_700_av
wdi_500_av
wdi_300_av
vwv_950_av
vwv_850_av
vwv_700_av
vwv_500_av
vwv_300_av
gp_950_av
gp_850_av
gp_700_av
gp_500_av
gp_300_av
gT_s_9_av
gT_9_8_av
gT_8_7_av
gT_7_5_av
gT_5_3_av
wSv_9_av
wSdir_9_av
wSv_7_av
wSdir_7_av
wSv_5_av
wSdir_5_av
wSv_1_av
wSdir_1_av
CBH_m_av
HigCC_p_av
LowCC_p_av
MidCC_p_av
TotCC_p_av
Cape_av
Cin_av
BLH_m_av
BLH_m_rt
LCL_hPa_av
LCL_m_av
LFC_hPa_av
CCL_hPa_av
EL_m_av
LiftIdx_av
VentIdx_av
CMLG_av
ros_p_lg1
f_start
geometry


In [28]:
gdf.to_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model.shp")

In [29]:
gdf_filtrado.to_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model_Outubro.shp")